In [ ]:
import os
import json

# Path to the gtFine folder
gtFine_folder = '/Users/mohitmb/Downloads/idd20kII/gtFine'

# Loop through all subfolders in the gtFine folder (train or val)
for split_folder in os.listdir(gtFine_folder):
    # Get the path to the subfolder
    split_folder_path = os.path.join(gtFine_folder, split_folder)
    
    # Check if the path is a directory
    if os.path.isdir(split_folder_path):
        # Loop through all JSON files in the subfolder
        for json_file in os.listdir(split_folder_path):
            if json_file.endswith('.json'):
                json_file_path = os.path.join(split_folder_path, json_file)

                # Load the JSON file
                with open(json_file_path, 'r') as f:
                    data = json.load(f)
                print(data['objects'][0]['label'])


In [ ]:
import json
import os
import numpy as np

gtFine_folder = '/Users/mohitmb/Downloads/idd20kII/gtFine'

# Define the class labels
class_labels = ['unlabeled', 'ego vehicle', 'rectification border', 'out of roi', 'static', 'dynamic', 'ground', 'road', 'sidewalk', 'parking', 'rail track', 'building', 'wall', 'fence', 'guard rail', 'bridge', 'tunnel', 'pole', 'polegroup', 'traffic light', 'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck', 'bus', 'caravan', 'trailer', 'train', 'motorcycle', 'bicycle']

# Create a dictionary to map pixel values to class labels
label_map = {}
for i in range(len(class_labels)):
    label_map[i] = class_labels[i]

# Loop through all subfolders in the gtFine folder
for split_folder in os.listdir(gtFine_folder):
    # Make sure the item is a directory
    if os.path.isdir(os.path.join(gtFine_folder, split_folder)):
        # Define the path to the current subfolder
        split_folder_path = os.path.join(gtFine_folder, split_folder)

        # Loop through all JSON files in the subfolder
        for json_file in os.listdir(split_folder_path):
            if json_file.endswith('.json'):
                json_file_path = os.path.join(split_folder_path, json_file)

                # Load the JSON file and do something with it
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    # do something with the json data

                seg_labels = np.zeros((data['imgHeight'], data['imgWidth']), dtype=np.uint8)
                
                # Loop through all objects in the JSON file and set the corresponding pixels to the class label
                for obj in data['objects']:
                    label = obj['label']
                    polygon = obj['polygon']
                    x_coords = [p[0] for p in polygon]
                    y_coords = [p[1] for p in polygon]
                    rr, cc = polygon2mask((data['imgHeight'], data['imgWidth']), (y_coords, x_coords))
                    seg_labels[rr, cc] = class_labels.index(label)
                
                # Save the segmentation label array as an image
                seg_img = Image.fromarray(seg_labels)
                seg_img.putpalette(np.array([
                    0, 0, 0,  # unlabeled
                    # RGB values for the other classes...
                ], dtype=np.uint8).flatten())
                seg_img.save(os.path.join(split_folder_path, 'segmentation', os.path.splitext(json_file)[0] + '.png'))


In [ ]:
import json

# Dictionary to map image file names to segmentation masks
mask_dict = {}

# Loop through all JSON files in the subfolder
for json_file in os.listdir(split_folder_path):
    if json_file.endswith('.json'):
        json_file_path = os.path.join(split_folder_path, json_file)

        # Load the JSON file and extract the segmentation mask
        with open(json_file_path) as f:
            json_data = json.load(f)
            mask = json_data['objects'][0]['polygon']

        # Map the mask to the image file name
        img_file_name = json_file.replace('_gtFine_polygons.json', '_leftImg8bit.png')
        mask_dict[img_file_name] = mask


In [ ]:
import json

# Loop through all JSON files in the subfolder
for json_file in os.listdir(split_folder_path):
    if json_file.endswith('.json'):
        json_file_path = os.path.join(split_folder_path, json_file)

        # Load the JSON file
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # Extract the class labels and segmentation masks
        image_id = data['imgName']
        labels = []
        masks = []
        for obj in data['objects']:
            label = obj['label']
            mask = obj['polygon']

            # Convert polygon to binary mask
            img_size = (data['imgWidth'], data['imgHeight'])
            mask_img = np.zeros(img_size, np.uint8)
            cv2.fillPoly(mask_img, [mask], 1)

            labels.append(label)
            masks.append(mask_img)

        # Save the labels and masks to a file
        output_file_path = os.path.join(output_folder, f'{image_id}.npz')
        np.savez(output_file_path, labels=labels, masks=masks)


In [ ]:
import os
import json

# Set the path to the folder containing the JSON files
gtFine_folder = '/Users/mohitmb/Downloads/idd20kII/gtFine'

# Set the name of the subfolder containing the JSON files for a particular split
split_folder = 'train'

# Set the path to the subfolder containing the JSON files
split_folder_path = os.path.join(gtFine_folder, split_folder)

# Loop through all JSON files in the subfolder
for json_file in os.listdir(split_folder_path):
    if json_file.endswith('.json'):
        json_file_path = os.path.join(split_folder_path, json_file)

        # Load the JSON file
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # Print the loaded JSON data
        print(data)
